## 块

为了实现复杂的网络，引入了神经网路块的概念，块可以表述单个层、多个层组成的主键或整个模型本身。
从编程的角度看，块由类（class）表示，它的任何子类都必须定义一个将输入转换为输出的正向传播函数，必须存储任何必须的参数。

In [1]:
# 多层感知机的列子
import torch
from torch import nn
from torch.nn import functional as F
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1886, -0.2589, -0.1761, -0.1069, -0.1637,  0.3169, -0.0077,  0.0686,
         -0.0025,  0.0075],
        [-0.1667, -0.2634, -0.1870, -0.1263, -0.1773,  0.1332,  0.0399,  0.0326,
         -0.0444,  0.0632]], grad_fn=<AddmmBackward>)

上面是一个多层感知机的例子，其中nn.Sequential和nn.Linear()是nn.Module的子类，它维护着Module组成的列表，正向传播将每个块的输出作为下个块的输入，其中net(X)是net.__call__(X)的简写。

## 自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__() # 调用父类的__init__()，完成初始化
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X))) # 前向传播 

In [3]:
net = MLP()
net(X)

tensor([[-0.1710, -0.1230, -0.0295, -0.1956, -0.2455, -0.3210, -0.0773,  0.2891,
         -0.0390,  0.1207],
        [-0.1216, -0.0678,  0.0289, -0.0873, -0.2657, -0.3352, -0.2335,  0.1411,
         -0.1627,  0.0787]], grad_fn=<AddmmBackward>)

## 顺序块
nn.Sequential是为了把模型串起来，我们只需要实现两个关键函数，即可实现nn.Sequential的功能：
1. 将块组个添加到列表中。
1. 将列表中的块串起来，实现正向传播函数。

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
#         self.blks = []
        for block in args:
            # 将block保存到self._module中，self._module的类型是OrderedDict，用list也行
            self._modules[block] = block
#             self.blks.append(block)
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X
    
#     def forward(self, X):
#         for block in self.blks:
#             X = block(X)
#         return X

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0213,  0.1711, -0.0348,  0.0355, -0.0842,  0.0574, -0.2455,  0.0144,
          0.3846,  0.0933],
        [-0.0819,  0.2342, -0.0391, -0.0323, -0.0899, -0.0722, -0.0190,  0.0531,
          0.4504,  0.1130]], grad_fn=<AddmmBackward>)

## 在前向传播函数中执行代码

前向传播函数中可以使用常数参数和python的控制流以实现不同的需求。

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的常数参数
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1) # 使用常数参数
        X = self.linear(X) # 复用全连接层
        
        while X.abs().sum() > 1: # 控制流
            X /= 2
        return X.sum()

In [7]:
net = FixedHiddenMLP()
net(X)

tensor(0.0423, grad_fn=<SumBackward0>)

## 混合使用块

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), 
                                 nn.Linear(64, 32), nn.ReLU())
        self.out = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.out(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1647, grad_fn=<SumBackward0>)